# Parsing Randomness
This is an example showing the principles of "Free Generators", a data structure both interpretable as:
- A Generator
- A Parser

First, like in the paper [TODO], the "original" Parser and Generator will be shown. Next, they will both be build using the introduced data structure `FreeGen`.

The utilised example are Binary Integer trees, first introduced here:

## Binary Integer trees
The trees will have the following elements:
- `N` stands for a node
  - A Node has 2 children (other tree)
  - and an Integer value assigned (`0..9`)
- `L` stands for leaf

An example could look like this:
```haskell
N: 2 
|- L
`- N: 1
   |- N: 6 -> L L
   `- L
```

A **Generator** creates such a data structure at random. A **Parser** gets some String input (for the shown example: `N 2 L N 1 N 6 L L L`) and creates the according instance of the data structure.


In [2]:
:reload
:load ./Definitions/Tree.hs

import Definitions.Tree

## Implementation of original Generator and Parser
To show the analogy of the structure in Generators and Parsers, this section will start with the original implementation of both for creating (either making or parsing) a Binary Boolean Tree.

Both the utilised Generator and Parser are already existing implementations from other libraries.

In [3]:
import Test.QuickCheck (Gen)
import qualified Test.QuickCheck.Gen as Gen

generateTree :: Int -> Gen Tree
generateTree 0 = return Leaf
generateTree h = do
  c <- Gen.frequency [(1, return 'L'), (3, return 'N')]
  case c of
    'L' -> return Leaf
    'N' -> do
      x <- generateInt
      l <- generateTree (h-1)
      r <- generateTree (h-1)
      return (Node x l r)

generateInt = Gen.oneof [ return x | x<-[0..9] ]

Gen.generate (generateTree 4)


N: 2
|- N: 6
|  |- N: 9
|  |  |- N: 6 -> L L
|  |  `- L
|  `- L
`- L

In [4]:
import Text.Parsec.String (Parser)
import Text.Parsec
import Data.Char

parseTree :: Int -> Parser Tree
parseTree 0 = return Leaf
parseTree h = do
  c <- oneOf "LN"
  case c of
    'L' -> return Leaf
    'N' -> do
      x <- parseInt
      l <- parseTree (h-1)
      r <- parseTree (h-1)
      return (Node x l r)

parseInt = digitToInt <$> digit

parse (parseTree 5) "" "N2LN1N6LLL"

Right 
N: 2
|- L
`- N: 1
   |- N: 6 -> L L
   `- L

## Implementation of Free Generator
As a first step, the general Free Generator for the Tree exampale will be defined. 

Important to notice: the resulting output `FreeGen Tree` is simply an abstract data structure. Without the according interpretation, it is neither a generator nor a parser. Still, the structure has a remarkable resemblence with both the definition to get a `Gen Tree` and a `Parser Tree`.

In [25]:
:reload
:load ./Definitions/FreeGen.hs

import Definitions.FreeGen

In [ ]:
freeGenTree :: Int -> FreeGen Tree
freeGenTree 0 = Return Leaf
freeGenTree h = Bind
  (Pick [(1, 'L', Return 'L'), (3, 'N', Return 'N')])
  (\c -> case c of
    'L' -> Return Leaf
    'N' -> Bind
      freeGenInt (\x -> Bind
      (freeGenTree (h-1)) (\l -> Bind
      (freeGenTree (h-1)) (\r ->
      Return (Node x l r)))))

freeGenInt = Pick [(1, intToDigit x, Return x) | x<-[0..9]]

-- Node about the output: the warnings can be ignored. 
-- The Code is written in the given way, to show the resemblance to the other definitions.